In [1]:
from scipy.io import arff
import pandas as pd
import sys
sys.path.append('/home/zh/opt/project/pycharmproject/minirocket/code')
from minirocket import fit, transform
from sklearn.linear_model import RidgeClassifierCV
import numpy as np
from sklearn.metrics import accuracy_score
import os
import csv
import time

/home/zh/.local/lib/python3.6/site-packages/numba/core/errors.py:154: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
# read data
X_Y_training_path = f"""/home/zh/opt/project/pycharmproject/minirocket/recurrence/data/DucksAndGeese/DucksAndGeese_TRAIN.arff"""
X_Y_test_path = f"""/home/zh/opt/project/pycharmproject/minirocket/recurrence/data/DucksAndGeese/DucksAndGeese_TEST.arff"""

# data format transform arff -> dataframe -> numpy.array
X_Y_training_data = arff.loadarff(X_Y_training_path)
X_Y_test_data = arff.loadarff(X_Y_test_path)

X_Y_training_df = pd.DataFrame(X_Y_training_data[0])
X_Y_training_target = X_Y_training_df['target']
X_Y_training_df = X_Y_training_df.drop(labels='target', axis=1)
X_Y_training_df = X_Y_training_df.sample(n=1024, random_state=1, axis=1)
X_Y_training_df['target'] = X_Y_training_target
# X_Y_training_df['target'] = X_Y_training_df['target'].apply(lambda x : x.decode('utf-8'))
# X_Y_training_df['target'] = X_Y_training_df['target'].map(dict(zip(X_Y_training_df['target'].unique(), range(X_Y_training_df['target'].nunique()))))

In [3]:
X_Y_training_df

,att135439,att15287,att138070,att111235,att235478,att54119,att12296,att98620,att156509,att170649,...,att187829,att190609,att233011,att157560,att167225,att66691,att156687,att104554,att196775,target
0,-0.009674,0.039888,0.017975,0.000336,0.010376,-0.012848,0.034181,-0.001770,0.006317,-0.047456,...,-0.026215,0.043031,0.035188,-0.001770,0.108890,-0.030549,0.021455,0.006775,-0.022462,b'Black-bellied_Whistling_Duck'
1,-0.072909,-0.007324,-0.007324,0.050294,0.035554,-0.027863,0.002960,-0.027070,-0.006958,-0.090396,...,-0.106479,0.177343,0.011078,-0.018067,0.019257,0.024415,0.030396,-0.013031,-0.031404,b'Black-bellied_Whistling_Duck'
2,-0.003784,-0.031648,0.008301,-0.092654,-0.043123,-0.002045,0.119694,-0.149419,-0.074618,0.096011,...,-0.007965,-0.038362,-0.069613,0.010529,0.017853,-0.000977,-0.112125,-0.006409,-0.021332,b'Black-bellied_Whistling_Duck'
3,0.047761,-0.013733,0.478591,0.026521,0.011719,-0.052583,0.004120,-0.015687,-0.054628,0.019807,...,0.020692,-0.004700,-0.008606,-0.059511,0.400220,-0.215186,0.071535,-0.034516,0.004791,b'Black-bellied_Whistling_Duck'
4,-0.004120,0.026093,-0.000031,-0.017396,-0.001495,0.012452,-0.005463,-0.009278,-0.000366,0.006775,...,-0.004975,-0.004242,0.002808,0.007508,0.004059,-0.002441,-0.005615,0.011902,-0.004822,b'Black-bellied_Whistling_Duck'
5,0.006073,-0.000366,-0.001801,0.007385,-0.023835,0.002472,0.000488,-0.000580,-0.004608,-0.001221,...,-0.005097,0.001831,0.004273,0.001679,-0.004944,-0.007080,0.002716,-0.004730,0.004456,b'Black-bellied_Whistling_Duck'
6,0.004608,0.003174,0.044130,-0.332713,-0.007263,0.002747,0.015870,-0.000153,0.001465,0.000305,...,0.001984,-0.013764,0.001862,-0.003235,0.005615,0.000977,-0.002686,0.014374,0.011231,b'Black-bellied_Whistling_Duck'
7,-0.016175,-0.041505,0.005005,-0.026337,-0.017823,0.026582,-0.012268,0.010956,-0.066866,-0.015229,...,-0.146672,0.027985,-0.040620,-0.129337,-0.030396,0.027589,-0.019410,-0.000580,-0.008362,b'Black-bellied_Whistling_Duck'
8,-0.021058,0.245766,0.060945,0.124271,-0.060274,-0.157628,-0.001648,-0.054079,-0.014924,0.024018,...,0.056429,-0.006500,-0.040803,0.070833,0.041932,-0.036653,-0.053468,-0.028382,-0.080630,b'Black-bellied_Whistling_Duck'
9,-0.019623,0.017701,-0.034608,0.035493,0.050111,-0.002686,0.002411,0.010773,-0.070132,0.002533,...,-0.005463,0.002625,-0.059633,-0.103977,0.010285,0.041505,-0.080935,0.002899,-0.038911,b'Black-bellied_Whistling_Duck'


In [4]:
X_Y_training_df.shape

(50, 1025)